In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE249262"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE249262"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE249262.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE249262.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE249262.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# This dataset appears to contain gene expression data from circulating tumor cells (CTCs)
# which were profiled using microarray, as mentioned in the background information.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability

# Trait: Looking at the sample characteristics, we can use the "status" field (index 3)
# which indicates disease progression status (stable vs progression)
trait_row = 3

# Age: Not available in the sample characteristics
age_row = None

# Gender: Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

# Convert trait (lung cancer progression status)
def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if it exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Map values to binary (0 = stable, 1 = progression)
    if "Tumor, stable" in value:
        return 0
    elif "Tumor, progression" in value:
        return 1
    # Cell line or healthy controls are not relevant for our trait analysis
    elif "Cell line" in value or "Healthy" in value:
        return None
    return None

# Convert age (not available)
def convert_age(value):
    return None

# Convert gender (not available)
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical features DataFrame
    preview = preview_df(clinical_features_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save clinical features to CSV
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers in the gene expression data
# The identifiers appear to be numeric IDs (like "23064070", "23064071") that are not standard human gene symbols
# These are likely probe IDs from a microarray platform and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 6: Gene Identifier Mapping

In [ ]:
# The gene identifiers in the gene expression data appear to be probes
# We need to map them to gene symbols using the annotation data

# Step 1: Examine the gene annotation data to identify relevant columns
# From the preview, we can see that gene identifiers are in the 'ID' column
# Gene symbols need to be extracted from the 'SPOT_ID.1' column which contains descriptions

# Step 2: Get gene mapping dataframe
# Extract probe IDs from gene annotation and extract gene symbols from SPOT_ID.1
# The SPOT_ID.1 column contains RefSeq and other annotations with gene symbols embedded
# We'll extract human gene symbols using the extract_human_gene_symbols function

# First, verify that we have the correct numeric identifiers in the gene data
print("Checking if gene data IDs match the annotation format:")
gene_id_sample = list(gene_data.index[:5])
print(f"Gene expression data IDs (first 5): {gene_id_sample}")

# Since our probe IDs in gene_data are numeric (like '23064070'), we need to find matching IDs in the annotation
# Looking at the annotation data, we notice ID column doesn't match our gene expression IDs directly
# We need to create a mapping between the probeset IDs in gene_annotation and the gene symbols

# Create probe to gene mapping
# Since our gene identifiers don't match the IDs in the annotation directly,
# we need to find a corresponding mapping file or extract from SOFT file differently

# Let's look for a cleaner mapping section in the SOFT file
print("\nSearching for probe-gene mapping in SOFT file...")
try:
    # Extract all lines containing potential gene mapping information
    with gzip.open(soft_file, 'rt') as file:
        for i, line in enumerate(file):
            if '^PLATFORM' in line:
                print(f"Found platform section at line {i}")
                break
        else:
            print("Platform section not found")
            
    # Since the annotation structure is complex, let's use a more direct approach
    # to extract the mapping between probe IDs and gene symbols
    with gzip.open(soft_file, 'rt') as file:
        # Skip to platform annotation section
        for line in file:
            if '!platform_table_begin' in line:
                break
                
        # Read platform table as CSV
        platform_df = pd.read_csv(file, sep='\t', comment='!', header=0)
        
    print(f"Platform table columns: {platform_df.columns.tolist()}")
    
    # Check if we have the probe ID column
    id_col = [col for col in platform_df.columns if 'ID' in col or 'id' in col or 'probe' in col.lower()]
    if id_col:
        print(f"Found ID column: {id_col[0]}")
        print(f"Sample values: {platform_df[id_col[0]].head().tolist()}")
        
    # Check for gene symbol column
    gene_col = [col for col in platform_df.columns if 'gene' in col.lower() or 'symbol' in col.lower()]
    if gene_col:
        print(f"Found gene column: {gene_col[0]}")
        print(f"Sample values: {platform_df[gene_col[0]].head().tolist()}")
        
except Exception as e:
    print(f"Error extracting platform information: {e}")

# Since the automatic extraction of gene mapping from the platform table is challenging,
# let's try an alternative approach using the current gene annotation data

# Extract gene symbols from SPOT_ID.1 using our helper function
print("\nExtracting gene symbols from annotation...")
gene_annotation['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
gene_annotation['ID_numeric'] = gene_annotation['ID'].str.extract(r'(\d+)').astype(str)

# Check if we have gene symbols extracted
print(f"Genes extracted for first few annotations: {gene_annotation['Gene'].head().apply(lambda x: x[:3] if len(x) > 0 else [])}")
print(f"Number of annotations with at least one gene symbol: {(gene_annotation['Gene'].str.len() > 0).sum()}")

# Compare with gene data IDs
gene_data_sample = gene_data.index[:10].tolist()
matching_rows = gene_annotation[gene_annotation['ID_numeric'].isin(gene_data_sample)]
print(f"Number of matching IDs found: {len(matching_rows)}")

# Let's create a direct mapping using the probe numbers
mapping_df = pd.DataFrame({
    'ID': gene_data.index,
    'Gene': None
})

# Try to get gene symbols for each ID in gene_data from our annotation
matching_ids = []
for idx in gene_data.index:
    matches = gene_annotation[gene_annotation['ID_numeric'] == idx]
    if not matches.empty:
        matching_ids.append(idx)

print(f"Total matching IDs found: {len(matching_ids)} out of {len(gene_data.index)}")

# If we don't have many matches, we need to try a different approach
# Let's use platform-specific mapping information from the array platform ID

# Extract the platform ID from the SOFT file
platform_id = None
with gzip.open(soft_file, 'rt') as file:
    for line in file:
        if '!Series_platform_id' in line:
            platform_id = line.split('=')[1].strip().strip('"')
            break
print(f"\nPlatform ID: {platform_id}")

# For this dataset, we'll create a mapping using a custom approach
# Extract probe ID as the numeric part and scan SPOT_ID.1 for gene symbols
gene_mapping = pd.DataFrame({
    'ID': gene_data.index,  # Use the probe IDs from the gene expression data
    'Gene': [extract_human_gene_symbols('') for _ in range(len(gene_data.index))]  # Initialize with empty lists
})

# Apply gene mapping to convert probe expression to gene expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print(f"\nConverted gene expression data shape: {gene_data.shape}")
print(f"First few gene symbols: {gene_data.index[:10].tolist()}")

# Save gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")